### Etapa 02

- **Módulo 01 de Detección de emociones con DeepFace**
- *DeepFace retorna:*
    - La emoción dominante.
    - Un diccionario con las probabilidades de cada emoción.
- **Módulo 02 de detección de atención (básica) con MediaPipe**
- *Métrica: EAR (Eye Aspect Ratio):*
    - Detectaremos cuánto están abiertos los ojos, lo cual se relaciona con atención o fatiga. El cálculo se puede hacer con los puntos de los ojos usando MediaPipe.

In [38]:
import os
import math
import cv2
import pandas as pd
from typing import Tuple, List
from deepface import DeepFace
import mediapipe as mp

In [39]:
def compute_ear(eye_points: List[Tuple[float, float]]) -> float:
    """
    Calcula el Eye Aspect Ratio (EAR) para estimar apertura ocular.
    """
    A: float = math.dist(eye_points[1], eye_points[5])
    B: float = math.dist(eye_points[2], eye_points[4])
    C: float = math.dist(eye_points[0], eye_points[3])
    return (A + B) / (2.0 * C)

In [40]:
def compute_ear(eye_points: List[Tuple[float, float]]) -> float:
    """
    Calcula el Eye Aspect Ratio (EAR) para estimar apertura ocular.
    """
    A: float = math.dist(eye_points[1], eye_points[5])
    B: float = math.dist(eye_points[2], eye_points[4])
    C: float = math.dist(eye_points[0], eye_points[3])
    return (A + B) / (2.0 * C)

In [41]:
def extract_ear_from_image(image_path: str) -> float:
    """
    Procesa una imagen para calcular el EAR usando MediaPipe.
    Retorna -1.0 si no se detecta rostro.
    """
    mp_face_mesh: mp.solutions.face_mesh = mp.solutions.face_mesh

    image: cv2.typing.MatLike = cv2.imread(image_path)
    if image is None:
        return -1.0  # Imagen inválida o no leída

    image_rgb: cv2.typing.MatLike = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1) as face_mesh:
        results = face_mesh.process(image_rgb)

        if not results.multi_face_landmarks:
            return -1.0

        face_landmarks = results.multi_face_landmarks[0]

        # Índices para el ojo derecho (MediaPipe)
        right_eye_indices: List[int] = [33, 160, 158, 133, 153, 144]
        height: int
        width: int
        height, width, _ = image.shape

        right_eye: List[Tuple[float, float]] = [
            (face_landmarks.landmark[i].x * width, face_landmarks.landmark[i].y * height)
            for i in right_eye_indices
        ]

        ear: float = compute_ear(right_eye)
        return round(ear, 3)

In [42]:
def classify_attention_from_ear(ear: float, threshold: float = 0.25) -> str:
    """
    Clasifica el estado de atención según el valor EAR.
    """
    if ear == -1.0:
        return "no_face"
    elif ear < threshold:
        return "drowsy"
    else:
        return "attentive"

In [43]:
def detect_emotion(image_path: str) -> Tuple[str, float]:
    """
    Usa DeepFace para detectar la emoción dominante en una imagen.
    Devuelve ('unknown', 0.0) si hay error.
    """
    try:
        result: dict = DeepFace.analyze(
            img_path=image_path,
            actions=["emotion"],
            enforce_detection=False
        )[0]

        emotion: str = result["dominant_emotion"]
        score: float = round(result["emotion"][emotion], 2)
        return emotion, score

    except Exception as e:
        print(f"⚠️ DeepFace error en '{image_path}': {e}")
        return "unknown", 0.0

***Pipeline Principal***

In [44]:
def analyze_video_frames(
    frames_folder: str,
    output_csv_path: str,
    attention_threshold: float = 0.25
) -> None:
    """
    Analiza todos los frames de una carpeta para emociones y atención.
    Guarda los resultados en un CSV.
    """
    results: List[dict] = []

    for filename in sorted(os.listdir(frames_folder)):
        if not filename.lower().endswith(".jpg"):
            continue

        frame_path: str = os.path.join(frames_folder, filename)

        # Detectar emoción con DeepFace
        emotion: str
        emotion_score: float
        emotion, emotion_score = detect_emotion(frame_path)

        # Calcular EAR y clasificar atención
        ear: float = extract_ear_from_image(frame_path)
        attention_status: str = classify_attention_from_ear(ear, threshold=attention_threshold)

        # Guardar resultado
        results.append({
            "frame": filename,
            "emotion": emotion,
            "emotion_score": emotion_score,
            "EAR": ear,
            "attention_status": attention_status
        })

    # Exportar resultados
    df: pd.DataFrame = pd.DataFrame(results)
    df.to_csv(output_csv_path, index=False, header=True)
    print(f"✅ Resultados guardados en '{output_csv_path}'")

analyze_video_frames("frames_output", "emotion_attention_results.csv")

W0000 00:00:1746557006.282565   68512 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746557006.287705   68512 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746557007.289534   68524 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746557007.293384   68524 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746557008.211360   68534 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746557008.216344   68534 inference_feedback_manager.cc:114] Feedback manager 

✅ Resultados guardados en 'emotion_attention_results.csv'


W0000 00:00:1746557023.990142   68754 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1746557023.993948   68754 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [45]:
df = pd.read_csv('emotion_attention_results.csv', header=0)
df.head(10)

,frame,emotion,emotion_score,EAR,attention_status
0,2025-04-25_12-26-31.jpg,neutral,60.30,0.270,attentive
1,2025-04-25_12-26-36.jpg,sad,75.49,0.263,attentive
2,2025-04-25_12-26-41.jpg,neutral,45.28,0.282,attentive
3,2025-04-25_12-26-46.jpg,fear,48.65,0.283,attentive
4,2025-04-25_12-26-51.jpg,neutral,38.52,0.164,drowsy
5,2025-04-25_12-26-56.jpg,fear,33.73,0.274,attentive
6,2025-04-25_12-27-01.jpg,neutral,88.38,0.287,attentive
7,2025-04-25_12-27-06.jpg,fear,56.12,0.244,drowsy
8,2025-04-25_12-27-11.jpg,fear,40.90,0.312,attentive
9,2025-04-25_12-27-16.jpg,sad,90.67,0.234,drowsy
